In [1]:
import gym
import tensorflow as tf

from stable_baselines.common.policies import MlpPolicy, CnnPolicy, MlpLnLstmPolicy, FeedForwardPolicy
from stable_baselines.common.vec_env import DummyVecEnv, VecCheckNan, SubprocVecEnv, VecFrameStack
from stable_baselines import PPO2, A2C


  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                         net_arch=[dict(pi=[128, 128, 128],
                                                        vf=[128, 128, 128])],
                                         feature_extraction="mlp")

def make_env(i):
    return lambda: gym.make('zrc_learn:zrc-v0')
env = DummyVecEnv([make_env(i) for i in range(1)])  # The algorithms require a vectorized environment to run
env = VecCheckNan(env, raise_exception=False)
#env = VecFrameStack(env, 4)

model = PPO2(CustomPolicy, env, n_steps=1500)
model.load('train')

print(model.act_model.obs_ph)
print(model.act_model.action)


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Tensor("input/Ob:0", shape=(?, 64, 11), dtype=float32)
Tensor("output/add:0", shape=(?, 3), dtype=float32)


In [ ]:
env.render()

scale = 10
for epoch in range(1000*scale): # 10,000 hours
    model.learn(total_timesteps=1200*scale) # 1 hour
    
    env.render()
    
    model.save('train')
    with model.graph.as_default():
        g = tf.graph_util.convert_variables_to_constants(model.sess, model.graph.as_graph_def(), ['output/add'])
        #tf.saved_model.save(g, '')
        tf.train.write_graph(g, '', 'test.pb', as_text=False)
        tf.saved_model.simple_save(model.act_model.sess, '', inputs={"obs": model.act_model.obs_ph}, outputs={"action": model.act_model.action})


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model.pb
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model.pb
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 

In [ ]:
builder = tf.saved_model.Builder('')
with model.graph.as_default():
        builder.add_meta_graph_and_variables(model.sess,
                                  ["serve"],
                                  signature_def_map={"act":tf.saved_model.build_signature_def(
    inputs={"obs": tf.saved_model.build_tensor_info(model.act_model.obs_ph)},
    outputs={"action": tf.saved_model.build_tensor_info(model.act_model.action)},
    method_name="act"
)})
builder.save()

In [4]:
#with model.graph.as_default():
    #g = tf.graph_util.convert_variables_to_constants(model.sess, model.graph.as_graph_def(), ['output/add'])
    #tf.saved_model.save(g, '')
    #tf.train.write_graph(g, '', 'test.pb', as_text=False)
tf.saved_model.simple_save(model.act_model.sess, '', inputs={"obs": model.act_model.obs_ph}, outputs={"action": model.act_model.action})


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model.pb


In [4]:
for epoch in range(1000*scale): # 10,000 hours
    obs = env.reset()
    reward = 0
    for i in range(120*scale): # 1 minute
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        reward += rewards
    print(reward)

KeyboardInterrupt: 